In [1]:
from keras.models import Sequential, load_model
from keras.layers import *
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, Callback
import keras

keras.backend.set_image_data_format('channels_last')

# telegram
import telegram
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math


import tensorflow as tf

# Importing matplotlib to plot images.
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# Importing SK-learn to calculate precision and recall
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneGroupOut, LeaveOneOut
from sklearn.utils import shuffle
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

# Used for graph export
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

import pickle as pkl
import h5py

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from pathlib import Path
import os.path
import sys
import datetime
import time

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
class LoggingTensorBoard(TensorBoard):    

    def __init__(self, log_dir, settings_str_to_log, **kwargs):
        super(LoggingTensorBoard, self).__init__(log_dir, **kwargs)

        self.settings_str = settings_str_to_log

    def on_train_begin(self, logs=None):
        TensorBoard.on_train_begin(self, logs=logs)

In [3]:
class TelegramCallback(Callback):

    def __init__(self, config):
        super(TelegramCallback, self).__init__()
        self.user_id = config['telegram_id']
        self.model_name = config['model_name']
        self.bot = telegram.Bot(config['token'])

    def send_message(self, text):
        try:
            self.bot.send_message(chat_id=self.user_id, text=text)
        except Exception as e:
            print('Message did not send. Error: {}.'.format(e))

    def on_train_begin(self, logs={}):
        text = 'Start training model {} | {}'.format(self.model.name, self.model_name)
        self.send_message(text)

    def on_epoch_end(self, epoch, logs={}):
        text = '{} | Epoch {}.\n'.format(self.model_name, epoch)
        for k, v in logs.items():
            text += '{}: {:.4f}; '.format(k, v)
        self.send_message(text)

In [4]:
pixel_width = {"N5X":1080,"S3Mini":480,"S4":1080,"N6":1440}
pixel_height = {"N5X":1920,"S3Mini":800,"S4":1920,"N6":2560}

def normalizeWidth(row):
    smartphone = row.Phone
    return row.XPress / pixel_width[smartphone]

def normalizeHeight(row):
    smartphone = row.Phone    
    return row.YPress / pixel_height[smartphone]

In [5]:
df = pd.read_pickle("../../data/step03.pkl")
#df.Sensor = df.Sensor.apply(lambda x: x[-80:-55])

In [6]:
def createDataSets(train_ids, test_ids):

    df_train = dfAll[dfAll.userID.isin(train_ids)]
    df_test = dfAll[dfAll.userID.isin(test_ids)]

    df_test = df_test.reset_index()
    df_train = df_train.reset_index()

    df_train2 = df_train[['Blobs', 'InputMethod']].copy()
    df_test2 = df_test[['Blobs', 'InputMethod']].copy()

    x_train = np.vstack(df_train2.Blobs)
    x_test = np.vstack(df_test2.Blobs)
    y_train = df_train2.InputMethod.values
    y_test = df_test2.InputMethod.values

    x_train = x_train.reshape(-1, 27, 15, 1)
    x_test = x_test.reshape(-1, 27, 15, 1)

    global num_classes
    num_classes = 2
    y_train_one_hot = utils.to_categorical(df_train2.InputMethod, num_classes)
    y_test_one_hot = utils.to_categorical(df_test2.InputMethod, num_classes)
    
    return (x_train, y_train_one_hot, x_test, y_test_one_hot)

In [7]:
df.XPress = df.apply(lambda x: normalizeWidth(x), axis=1)
df.YPress = df.apply(lambda x: normalizeHeight(x), axis=1)

df.Sensor = df.Sensor.apply(lambda x: x.reshape(-1, 6, 1))

shape = df.Sensor.iloc[0].shape

In [8]:
def createDataSet(train_ids, test_ids):
    np.random.seed(42)
    np.random.shuffle(train_ids)

    dfTrain = df[df.PId.isin(train_ids)]
    dfTest = df[df.PId.isin([test_ids])]

    train_x = np.concatenate(dfTrain.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
    train_y = dfTrain[['XPress', 'YPress']].values
    train_x = np.concatenate([train_x,train_x,train_x,train_x,train_x,train_x,train_x,train_x])
    train_y = np.concatenate([train_y,train_y,train_y,train_y,train_y,train_y,train_y,train_y])

    test_x = np.concatenate(dfTest.Sensor.values).reshape(-1, shape[0], shape[1], shape[2])
    test_y = dfTest[['XPress', 'YPress']].values
    return (train_x, train_y, test_x, test_y)

In [ ]:
lst = df.PId.unique()
readable_timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y%m%d_%H%M%S')

for test_PID in todo_lst:
    test_pids = np.array(test_PID)
    train_pids = np.setdiff1d(lst, [test_PID])
    print("Train", train_pids, "\tTest", test_pids)

    train_x, train_y, test_x, test_y = createDataSet(train_pids, test_pids)

    # If GPU is not available: 
    # GPU_USE = '/cpu:0'
    #config = tf.ConfigProto(device_count = {"GPU": 1})


    # If GPU is available: 
    config = tf.ConfigProto()
    config.log_device_placement = True
    config.allow_soft_placement = True
    config.gpu_options.allocator_type = 'BFC'
    config.gpu_options.allow_growth=True

    # Limit the maximum memory used
    config.gpu_options.per_process_gpu_memory_fraction = 0.2

    # set session config
    tf.keras.backend.set_session(tf.Session(config=config))

    batch_size = 64

    tf.get_default_graph()
    model = Sequential()
    model.add(Conv2D(filters = 32, kernel_size = (3,3),padding = 'Same', activation ='relu', input_shape = (shape)))
    model.add(BatchNormalization(axis=-1))
    model.add(MaxPool2D(pool_size=(2,1), padding='Same'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(64, activation = "relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))

    model.add(Dense(16, activation = "relu"))
    model.add(BatchNormalization(axis=-1))
    model.add(Dropout(0.5))

    model.add(Dense(2, activation = "relu"))


    ## Compiling the model to generate a model
    def eucInMM(y_true, y_pred):
        return (K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))) * constant_pixels
    def euc(y_true, y_pred):
        return K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1, keepdims=True))#
    def eucStdMM(y_true, y_pred):
        return K.std(K.sqrt(K.sum(K.square(y_true - y_pred), axis=-1))) * constant_pixels
    def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_true - y_pred)))

    optimizer = optimizers.Adam(epsilon=1e-08)
    model.compile(loss="mse", optimizer=optimizer, metrics=[rmse, euc])

    # Broadcast progress to the tensorboard.
    file_name = "IMU_general-cross-" + str(test_PID) + "_" + readable_timestamp
    tensorboardFolder = "/srv/share/tensorboardfiles/" + file_name

    print(file_name)
    #model.summary()

    callbacks = [
        ModelCheckpoint(str(Path.home()) + "/data/cross/"+ file_name + ".h5",
                        monitor='val_eucInMM',
                        verbose=0,
                        save_best_only=False,
                        save_weights_only=False
                       ),
        LoggingTensorBoard(settings_str_to_log=config,
                           log_dir=tensorboardFolder,
                           histogram_freq=0,
                           write_graph=True,
                           write_images=True,
                           update_freq='epoch'
                          ),
        ReduceLROnPlateau(monitor='val_loss',
                          patience=10,
                          verbose=1,
                          factor=0.9,
                          min_lr=0.00001),
        TelegramCallback(
            {
                'token': "707020808:AAHJ22G0diUExRP4JUITk1VKuPtBPUVQ-J4",
                'telegram_id': -320890356,
                'model_name': file_name,
            }
        )
    ]

    #history = model.fit_generator(myGenerator("train", batch_size),
    #                              steps_per_epoch=len(train_x) // batch_size,
    #                              epochs=1000,
    #                              verbose=1,
    #                              validation_data=myGenerator("test", batch_size),
    #                              validation_steps=len(test_x) // batch_size,
    #                              callbacks=callbacks
    #                             )

    history = model.fit(x = train_x, 
                        y = train_y, 
                        batch_size = batch_size,
                       epochs=250,
                       verbose=2,
                       validation_data=(test_x, test_y),
                       callbacks=callbacks,
                       )
    score = model.evaluate(test_x, test_y, verbose=2)
    print('%s: %.2f' % (model.metrics_names[1], score[1]))
    print('Baseline Error: %.2f' %(1 - score[1]))

Train [ 1  2  3  4  5  6  7  9 10 11 12 13 14 15 16 17 18 19 20] 	Test 8
IMU_general-cross-8_20190201_214352
Train on 262656 samples, validate on 1728 samples
Epoch 1/250
